##### 데이터 설명
- 항공기 연착 확률 예측
- 데이터는 인천공항을 기준으로 수집

##### 컬럼 설명
- Date: 날짜
- Plan: 계획된 비행 대수 = Flight + IROPS
- Flight: 계획된 시간에 도착하거나 출발한 대수 = Plan - IROPS => 얘도 삭제
- IROPS: 비정상운항 (계획된 시간에 도착 또는 출발을 못함) = Plan - Flight
- AWS: 공항의 평균 풍속
- MWS: 공항의 최대 풍속
- IROPS_Ratio_Plan: 계획된 비행 기준으로 비정상운항 비율 = (IROPS / Plan) * 100
- IROPS_Ratio_Flight: 운행된 비행 기준으로 비정상운항 비율 = (IROPS / Flight) * 100 => 안쓸거라 삭제함

In [141]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df = pd.read_csv('data/인천공항연착확률예측.csv', index_col=0) # 인덱스 설정 반드시 할 것
df

,Plan,IROPS,AWS,MWS,IROPS_Ratio_Plan
Date,,,,,
20210101,310,46,5.5,13,14.838710
20210102,286,23,8.3,15,8.041958
20210103,291,38,4.4,11,13.058419
20210104,257,33,5.3,15,12.840467
20210105,319,20,9.3,17,6.269592
20210106,362,76,9.5,33,20.994475
20210107,419,149,17.5,28,35.560859
20210108,392,72,9.7,18,18.367347
20210109,372,60,7.7,15,16.129032


In [142]:
df.corr()

,Plan,IROPS,AWS,MWS,IROPS_Ratio_Plan
Plan,1.000000,0.504096,0.384496,0.183085,0.277457
IROPS,0.504096,1.000000,0.626892,0.528000,0.963074
AWS,0.384496,0.626892,1.000000,0.752123,0.549158
MWS,0.183085,0.528000,0.752123,1.000000,0.520337
IROPS_Ratio_Plan,0.277457,0.963074,0.549158,0.520337,1.000000


In [143]:
# 데이터 전처리

# 날짜를 인덱스로 설정하지 않으면 여기서 스케일링할 때 날짜도 같이 들어감
scaler = StandardScaler()
scaled = scaler.fit_transform(df)
df_scaled = pd.DataFrame(scaled, columns=df.columns, index=df.index) # 스케일링한 데이터 데이터프레임으로 변경 / columns, index 옵션으로 원본 df와 컬럼, 인덱스 일치시킴
df_scaled

,Plan,IROPS,AWS,MWS,IROPS_Ratio_Plan
Date,,,,,
20210101,-0.913781,-0.405491,-0.658164,-0.486109,-0.161733
20210102,-1.394192,-1.220016,0.143079,-0.150862,-1.199263
20210103,-1.294106,-0.688804,-0.972939,-0.821357,-0.433496
20210104,-1.974688,-0.865875,-0.715396,-0.150862,-0.466767
20210105,-0.733627,-1.326258,0.429238,0.184386,-1.469816
20210106,0.127109,0.656932,0.486469,2.866369,0.777949
20210107,1.268084,3.242161,2.775737,2.028249,3.001520
20210108,0.727622,0.515275,0.543701,0.352010,0.376916
20210109,0.327280,0.090306,-0.028616,-0.150862,0.035235


In [144]:
# 학습, 테스트 데이터 만들기

x_data = np.array(df_scaled[['Plan', 'IROPS', 'AWS', 'MWS']]) # 스케일링한 데이터프레임에서 피쳐 가져옴 2차원 배열
y_data = np.array(df_scaled['IROPS_Ratio_Plan']) # 스케일링한 데이터프레임에서 예측값 가져욤

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, train_size=0.8, random_state=100, shuffle=True) # 훈련, 테스트 데이터 나눠줌 / random_state 줘서 고정

In [145]:
# 모델 생성, 학습, 평가

model = LinearRegression()
model.fit(x_train, y_train)
score = model.score(x_test, y_test)
print('점수: ', score)

점수:  0.9353479830729117


In [163]:
# 모델 활용

plan = int(input('비행계획 대수: '))
irops = int(input('비정상운항 대수: '))
aws = float(input('평균 풍속: '))
mws = float(input('최대 풍속: '))

features = np.array([plan, irops, aws, mws]).reshape(1, -1)

predict = model.predict(features)
print('연착 발생확률: ', predict)

# 피지컬 이용한 모델 활용
# 실시간으로 예측을 하여 LED 빨 노 초 로 상태표시
# 지금 GPIO 그런거 못쓰니까 그냥 문자열 출력으로 대체함

# 기준
# 0 ~ 30% 는 초록불 양호
# 31 ~ 70% 는 노란불 보통
# 71 이상은 빨간불 연착예상

if predict > 70 :
  print("LED RED : 연착예상")
elif predict <= 70 and predict > 30 :
  print("LED YELLOW : 보통")
elif predict <= 30 :
  print("LED GREEN : 양호")

연착 발생확률:  [6.54138801]
LED GREEN : 양호


- 데이터를 더 추가해서 정확도를 높여야할지...
- 혹시 입력받은 데이터를 스케일링해서 예측을 할 수는 없는지..
